In [1]:
import pandas as pd
csv_file='./data/test/test.csv'
df = pd.read_csv(csv_file)

In [2]:
import utils
import torch
from model.data_loader import DataLoader

json_path='./experiments/esm_650M/params.json'
params = utils.Params(json_path)
params.cuda = torch.cuda.is_available()
data_loader = DataLoader('test', params)


In [3]:
seq_embed,seq_lable=data_loader.extract_embeddings(df['label'],df['seq'])

Processing batch 1 of 396
Processing batch 2 of 396
Processing batch 3 of 396
Processing batch 4 of 396
Processing batch 5 of 396
Processing batch 6 of 396
Processing batch 7 of 396
Processing batch 8 of 396
Processing batch 9 of 396
Processing batch 10 of 396
Processing batch 11 of 396
Processing batch 12 of 396
Processing batch 13 of 396
Processing batch 14 of 396
Processing batch 15 of 396
Processing batch 16 of 396
Processing batch 17 of 396
Processing batch 18 of 396
Processing batch 19 of 396
Processing batch 20 of 396
Processing batch 21 of 396
Processing batch 22 of 396
Processing batch 23 of 396
Processing batch 24 of 396
Processing batch 25 of 396
Processing batch 26 of 396
Processing batch 27 of 396
Processing batch 28 of 396
Processing batch 29 of 396
Processing batch 30 of 396
Processing batch 31 of 396
Processing batch 32 of 396
Processing batch 33 of 396
Processing batch 34 of 396
Processing batch 35 of 396
Processing batch 36 of 396
Processing batch 37 of 396
Processing

In [4]:
batch_data, batch_labels = torch.FloatTensor(seq_embed), torch.LongTensor(seq_lable)
batch_data, batch_labels = batch_data.cuda(), batch_labels.cuda()

/tmp/ipykernel_36628/111880977.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  batch_data, batch_labels = torch.FloatTensor(seq_embed), torch.LongTensor(seq_lable)


In [5]:
import model.net as net
import os
model = net.Net(params).cuda() if params.cuda else net.Net(params)
utils.load_checkpoint('./experiments/esm_650M/best.pth.tar', model)
model.eval()

Net(
  (layer1): Linear(in_features=1280, out_features=640, bias=True)
  (layer2): Linear(in_features=640, out_features=12, bias=True)
)

In [6]:
output_batch = model(batch_data)

In [11]:
from sklearn.metrics import accuracy_score

num_classes=12
batch_labels=batch_labels.cpu()
y_val_pred = output_batch.cpu().argmax(dim=1).numpy()
# 计算每个类别的准确率
class_accuracies = []
for class_label in range(num_classes):
    class_indices = (y_val_pred == class_label)
    class_accuracy = accuracy_score(batch_labels[class_indices], y_val_pred[class_indices])
    class_accuracies.append(class_accuracy)
    print(f'Class {class_label} Accuracy: {class_accuracy}')

# 计算总体准确率
overall_accuracy = accuracy_score(batch_labels, y_val_pred)
print(f'Overall Accuracy: {overall_accuracy}')

Class 0 Accuracy: 0.9993181043300375
Class 1 Accuracy: 0.9980815347721822
Class 2 Accuracy: 0.9994890137966275
Class 3 Accuracy: 1.0
Class 4 Accuracy: 0.9939903846153846
Class 5 Accuracy: 0.9953703703703703
Class 6 Accuracy: 0.9987468671679198
Class 7 Accuracy: 0.9986631016042781
Class 8 Accuracy: 1.0
Class 9 Accuracy: 1.0
Class 10 Accuracy: 0.9857142857142858
Class 11 Accuracy: 0.9365079365079365
Overall Accuracy: 0.9977622745820719


In [39]:
import torch.nn.functional as F

In [40]:
batch_labels

tensor([ 1,  1,  1,  1,  1,  1,  1,  5,  1,  1,  3,  3,  5,  5,  5,  5,  3,  3,
         3,  5,  5,  3,  4,  3,  5,  3,  3,  6,  4,  4,  5,  4,  4,  4,  5,  4,
         4,  3,  0,  4,  4,  6,  1,  6, 10,  6,  6,  0,  0,  0,  6,  0,  0,  0,
         0,  6, 11,  6,  6,  6, 10, 10,  0,  0,  7,  7,  7,  7,  7,  7, 12,  7,
         7, 10, 10, 10, 12,  7, 12, 10, 11, 12,  7, 12, 12, 12,  2, 12, 12,  9,
        12,  9,  9,  2,  9,  9,  9,  2,  8,  2,  2,  8,  2,  9,  9,  9, 11,  8,
         2,  2,  9,  2, 11, 11,  2,  8, 11, 11, 11, 11, 11,  8, 10, 10, 10,  8,
         8,  8,  8,  8], device='cuda:0')

In [92]:
for index in range(0,len(output_batch)):
    softmax_output = F.softmax(output_batch[index], dim=0)
    #if batch_labels[index].item() == 0:
        # print(str(batch_labels[index].item()+1)," | ".join([f"cas {i+1}: {probability.item():.4f}" for i, probability in enumerate(softmax_output)]))
    all_data=["{:.2%}".format(probability) for i, probability in enumerate(softmax_output)]
    print("cas"+str(batch_labels[index].item()+1)+',',','.join(all_data))

cas2, 0.00%,99.99%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
cas2, 0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
cas2, 0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
cas2, 0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
cas2, 0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
cas2, 0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
cas2, 0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
cas6, 0.07%,0.00%,0.00%,0.00%,0.00%,99.93%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
cas2, 0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
cas2, 0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
cas4, 0.00%,0.00%,0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
cas4, 0.00%,0.00%,0.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
cas6, 0.00%,0.00%,0.00%,0.00%,0.00%,100.00

In [45]:
_, index = torch.max(output_batch, 1)

In [46]:
percentage = torch.nn.functional.softmax(output_batch, dim=1) * 100

In [48]:
_, indices = torch.sort(output_batch, descending=True)
[(batch_labels[idx], percentage[idx].item()) for idx in indices[0][:5]]

RuntimeError: a Tensor with 12 elements cannot be converted to Scalar